In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import librosa
import os
from sklearn.preprocessing import LabelBinarizer
import datetime

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def load_audio_file(file_path):
    input_length = 16000
    data = librosa.core.load(file_path, sr=16000)[0]
    if len(data) >= input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, 16000 - len(data)), "constant", constant_values=0)
    return data

In [3]:
target_label = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "unknown", "silence"]
valid_labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence"]

lb = LabelBinarizer().fit(target_label)

bg_path = './train/audio/_background_noise_/'
bg_list = os.listdir(bg_path) 
bg_list.remove('README.md')

audio_path = './train/audio/'
labels = os.listdir(audio_path)

train_path = "./tfrecord/train_wav.tfrecord"

writer = tf.python_io.TFRecordWriter(train_path)

In [5]:
for label in labels: 
    if label == '.DS_Store': continue
    else:
        orig_label = label
        file_path = audio_path + label + "/" 
        files = os.listdir(file_path) 

        if label not in valid_labels:
            label = "unknown"

        encoded_label = lb.transform([label])[0] 

        cnt = 0

        for file in files[:10]:
            if file == '.DS_Store' or file == 'README.md': continue
            else:
                filename = file_path + file
                
                signal = load_audio_file(filename)

                feature = {
                    "spectrum": float_feature(signal),
                    "label": int64_feature(encoded_label)
                }

                features = tf.train.Features(feature=feature)
                example = tf.train.Example(features=features)
                writer.write(example.SerializeToString())
                
                cnt += 1
            
        print("Writing {} is complete. ({})".format(orig_label, cnt))

writer.close()

Writing _background_noise_ is complete. (6)
Writing bed is complete. (10)
Writing bird is complete. (10)
Writing cat is complete. (10)
Writing dog is complete. (10)
Writing down is complete. (10)
Writing eight is complete. (10)
Writing five is complete. (10)
Writing four is complete. (10)
Writing go is complete. (10)
Writing happy is complete. (10)
Writing house is complete. (10)
Writing left is complete. (10)
Writing marvin is complete. (10)
Writing nine is complete. (10)
Writing no is complete. (10)
Writing off is complete. (10)
Writing on is complete. (10)
Writing one is complete. (10)
Writing right is complete. (10)
Writing seven is complete. (10)
Writing sheila is complete. (10)
Writing silence is complete. (10)
Writing six is complete. (10)
Writing stop is complete. (10)
Writing three is complete. (10)
Writing tree is complete. (10)
Writing two is complete. (10)
Writing up is complete. (10)
Writing wow is complete. (10)
Writing yes is complete. (10)
Writing zero is complete. (10)

In [7]:
def parser(serialized_example):
    features = {
        "spectrum": tf.FixedLenFeature([16000], tf.float32),
        "label": tf.FixedLenFeature([12], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    
    spec = parsed_feature['spectrum']
    label = parsed_feature['label']
    
    return spec, label

tf.reset_default_graph()

train_data_dir = "./tfrecord/train_wav.tfrecord"

train_dataset = tf.data.TFRecordDataset(train_data_dir).map(parser)
train_dataset = train_dataset.shuffle(10)
train_dataset = train_dataset.batch(10)

itr = tf.contrib.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)

spec, label = itr.get_next()

# spec = tf.reshape(spec, [128, 32]) # conv를 위해 depth도 추가해줘야 한다.
spec = tf.cast(spec, tf.float32)

train_init_op = itr.make_initializer(train_dataset)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    sess.run(train_init_op)
    for test_step in range(9999999999):
        try:
            _specs, _labels = sess.run([spec, label])
            
            for _spec in _specs:
                print(_spec.shape)

#             _spec = np.asarray(_spec[0])
#             librosa.display.specshow(_spec)
#             plt.show()
   
        except tf.errors.OutOfRangeError:
            break

(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(16000,)
(

In [ ]:
class PreProcessing:
    def __init__(self, signal_array, bg_path):
        self.signal_array = signal_array
        self.bg_path = bg_path
        self.bg_list = os.listdir(bg_path)
        
    def get_spectdata(wav, sr=16000, size=12800):
        spect = librosa.feature.melspectrogram(wav, sr=sr, hop_length=161, n_fft=2048)  
        log_spect = librosa.core.logamplitude(spect)
        data = np.asarray(log_spect).reshape(size)
        return data, log_spect
    
    def speed_tuning(wav): 
        speed_rate = np.random.uniform(0.9, 1.1)
        wav_speed_tune = cv2.resize(wav, (1, int(len(wav) * speed_rate))).squeeze()
        
        if len(wav_speed_tune) < 16000: 
            pad_len = 16000 - len(wav_speed_tune)
            wav_speed_tune = np.r_[np.random.uniform(-0.001, 0.001, int(pad_len / 2)),
                                   wav_speed_tune,
                                   np.random.uniform(-0.001, 0.001, int(np.ceil(pad_len / 2)))]
        else: 
            cut_len = len(wav_speed_tune) - 16000
            wav_speed_tune = wav_speed_tune[int(cut_len / 2): int(cut_len / 2) + 16000]
        return wav_speed_tune
    
    def pitch_tuning(wav, sample_rate=16000):
        bins_per_octave = 24 
        pitch_pm = 4
        pitch_change = pitch_pm * 2 * (np.random.uniform() - 0.5)
    
        wav_pitch_changed = librosa.effects.pitch_shift(wav.astype('float64'), 
                                                        sample_rate, 
                                                        n_steps=pitch_change, 
                                                        bins_per_octave=bins_per_octave)
        return wav_pitch_changed
    
    def bg_mixing(wav, bg): 
        start_ = np.random.randint(bg.shape[0] - 16000)
        bg_slice = bg[start_ : start_ + 16000]
        wav_with_bg = wav * np.random.uniform(0.8, 1.2) + bg_slice * np.random.uniform(0, 0.1)
        return wav_with_bg

    def choice_bg(self):
        return librosa.load(self.bg_path + self.bg_list[np.random.randint(0, 5)], sr=None)[0]
    
    def make_output(self):
        aug_output = []
        
        for i in range(len(self.signal_array)):
            signal_dict = {}
            
            signal = self.signal_array[i]
            
            signal_dict[0] = signal
 
            signal_dict[1] = self.speed_tuning(signal)

            signal_dict[2] = self.pitch_tuning(signal)

            bg = self.choice_bg()
            signal_dict[3] = self.bg_mixing(signal, bg)

            bg = self.choice_bg()
            signal_dict[4] = self.bg_mixing((self.speed_tuning(signal)), bg)

            signal_dict[5] = self.speed_tuning(self.pitch_tuning(signal))

            bg = self.choice_bg()
            signal_dict[6] = self.bg_mixing(self.pitch_tuning(signal), bg)

            bg = self.choice_bg()
            signal_dict[7] = self.bg_mixing(self.speed_tuning(self.pitch_tuning(signal)), bg)
            
            aug_signal = [self.get_spectdata(signal_dict[j])[0] for j in signal_dict.keys()]
            
            aug_output.append(aug_signal)
            
        return aug_output